In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
import scanpy_scripts as ss
import bbknn
import phate

np.set_printoptions(linewidth=180)
sc.settings.verbosity = 1
expr_cmap = ss.lib.expression_colormap()
ss.lib.set_figsize((4, 4))

In [23]:
new_ads = {}
for ct in ['endothelium', 'keratinocytes', 'melanocytes', 'neuronal', 'stroma']:
    print(ct)
    ad = sc.read(f'pooled_{ct}.processed.h5ad')
    rachel_annot = 'rachel_annot3' if 'rachel_annot3' in ad.obs.columns else 'rachel_annot2' if 'rachel_annot2' in ad.obs.columns else 'rachel_annot1'
    new_ad = anndata.AnnData(
        X=ad.raw.X,
        obs=ad.obs[[
            'batch', 'chemistry_sorting', 'dataset', 'day', 'donor', 'gender', 'nh3_annot1', rachel_annot, 'sample_id', 'strain', 'week', 'annot',
            'n_counts', 'n_genes', 'percent_hb', 'percent_mito', 'percent_ribo', 'percent_top50',
            'leiden_hm_r0_1', 'leiden_hm_r0_3', 'leiden_hm_r0_5', 'leiden_hm_r0_7', 'leiden_hm_r0_9'
        ]].copy(),
        var=ad.var[['highly_variable', 'means', 'dispersions', 'dispersions_norm']].copy(),
        obsm=ad.obsm
    )
    del new_ad.obsm['X_pca'], new_ad.obsm['X_pca_hm']
    new_ads[ct] = new_ad

neuronal


In [26]:
new_ads

{'keratinocytes': AnnData object with n_obs × n_vars = 4036 × 15277 
     obs: 'batch', 'chemistry_sorting', 'dataset', 'day', 'donor', 'gender', 'nh3_annot1', 'rachel_annot3', 'sample_id', 'strain', 'week', 'annot', 'n_counts', 'n_genes', 'percent_hb', 'percent_mito', 'percent_ribo', 'percent_top50', 'leiden_hm_r0_1', 'leiden_hm_r0_3', 'leiden_hm_r0_5', 'leiden_hm_r0_7', 'leiden_hm_r0_9'
     var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
     obsm: 'X_umap_hm',
 'melanocytes': AnnData object with n_obs × n_vars = 2184 × 13449 
     obs: 'batch', 'chemistry_sorting', 'dataset', 'day', 'donor', 'gender', 'nh3_annot1', 'rachel_annot3', 'sample_id', 'strain', 'week', 'annot', 'n_counts', 'n_genes', 'percent_hb', 'percent_mito', 'percent_ribo', 'percent_top50', 'leiden_hm_r0_1', 'leiden_hm_r0_3', 'leiden_hm_r0_5', 'leiden_hm_r0_7', 'leiden_hm_r0_9'
     var: 'highly_variable'
     obsm: 'X_umap_hm',
 'neuronal': AnnData object with n_obs × n_vars = 13350 × 18812 
     

In [24]:
for ct in ['endothelium', 'keratinocytes', 'melanocytes', 'neuronal', 'stroma']:
    new_ads[ct].write(f'pooled_{ct}.cellxgene.h5ad', compression='lzf')